In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth

In [5]:
df = pd.read_csv('./datasets/driver_details.csv')

In [7]:
df.shape

(4000, 3)

In [8]:
df.isna().sum()

Driver_ID           0
Distance_Feature    0
Speeding_Feature    0
dtype: int64

In [9]:
df[df.isnull().any(axis=1)]

,Driver_ID,Distance_Feature,Speeding_Feature


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Driver_ID,4000.0,3.423312e+09,1154.844867,3.423310e+09,3.423311e+09,3.423312e+09,3.423313e+09,3.423314e+09
Distance_Feature,4000.0,7.604152e+01,53.469563,1.552000e+01,4.524750e+01,5.333000e+01,6.563250e+01,2.447900e+02
Speeding_Feature,4000.0,1.072100e+01,13.708543,0.000000e+00,4.000000e+00,6.000000e+00,9.000000e+00,1.000000e+02


In [12]:
df.head(2)

,Driver_ID,Distance_Feature,Speeding_Feature
0,3423311935,71.24,28
1,3423313212,52.53,25


In [13]:
features = df.drop(columns=['Driver_ID'])

In [17]:
parameters = {
    'n_clusters': [2, 3, 4, 5, 10, 20, 30]
}
parameter_grid = ParameterGrid(parameters)
# [[{k: y} for y in x] for k, x in parameters.items()]

In [19]:
list(parameter_grid)

[{'n_clusters': 2},
 {'n_clusters': 3},
 {'n_clusters': 4},
 {'n_clusters': 5},
 {'n_clusters': 10},
 {'n_clusters': 20},
 {'n_clusters': 30}]